### Import Required modules and initialize SparkSession

In [1]:
from __future__ import print_function
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Using Split Functions").getOrCreate()

In [2]:
spark

In [3]:
data = [
  ('user1', 'Amaravati,AP,India'),
  ('user2', 'Karnataka,India'),
  ('user3', 'Hyderabad,Telangana,India'),
  ('user4', 'India'),
]

In [4]:
df = spark.createDataFrame(data, ['cust_name', 'full_address'])
df.show()

+---------+--------------------+
|cust_name|        full_address|
+---------+--------------------+
|    user1|  Amaravati,AP,India|
|    user2|     Karnataka,India|
|    user3|Hyderabad,Telanga...|
|    user4|               India|
+---------+--------------------+



In [5]:
import pyspark.sql.functions as f
df1 = (
  df.select(
    f.col("cust_name"),
    f.reverse(f.split(df.full_address,",")).alias("address")
  )
)

df1.printSchema()
df1.show(truncate = False)

root
 |-- cust_name: string (nullable = true)
 |-- address: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---------+-----------------------------+
|cust_name|address                      |
+---------+-----------------------------+
|user1    |[India, AP, Amaravati]       |
|user2    |[India, Karnataka]           |
|user3    |[India, Telangana, Hyderabad]|
|user4    |[India]                      |
+---------+-----------------------------+



In [6]:
result_df = df1.select (
  f.col("cust_name"),
  (f.col("address")[2]).alias("city"),
  (f.col("address")[1]).alias("state"),
  (f.col("address")[0]).alias("country"),
)

result_df.printSchema()
result_df.show(truncate=False)

root
 |-- cust_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)

+---------+---------+---------+-------+
|cust_name|city     |state    |country|
+---------+---------+---------+-------+
|user1    |Amaravati|AP       |India  |
|user2    |null     |Karnataka|India  |
|user3    |Hyderabad|Telangana|India  |
|user4    |null     |null     |India  |
+---------+---------+---------+-------+

